In [1]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import keras
from keras import np_utils

In [2]:
data=pd.read_csv('C:/Users/USER/Documents/dataset/lang.csv')

In [3]:
data['Text']=data["Text"].str.lower()
x=data['Text']
y=data['language']


In [4]:
def cleanLower(texts):
    
    # This regular expression pattern means everything except alphabetical characters
    pattern = "^[a-zA-Z]"
    cleanText = []
    for text in texts:
        # re.sub(pattern) means replace everything with a space except alphabetical characters
        cleanText.append(re.sub(pattern," ",text).lower())
        
    return cleanText

x = cleanLower(x)


In [6]:
x[1]

' ebes joseph pereira thomas  på eng the jesuits and the sino-russian treaty of nerchinsk  the diary of thomas pereira bibliotheca instituti historici s i --   rome libris '

In [33]:
y

0        Estonian
1         Swedish
2            Thai
3           Tamil
4           Dutch
           ...   
21995      French
21996        Thai
21997     Spanish
21998     Chinese
21999    Romanian
Name: language, Length: 22000, dtype: object

In [158]:

max_words = 50000
max_len = 100

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(list(data['Text']))
train_data = tokenizer.texts_to_sequences(list(data['Text']))

In [159]:
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data,maxlen = max_len)

In [160]:
train_data

array([[    0,     0,     0, ...,  2170,    82,  3638],
       [    0,     0,     0, ...,    16,  4068, 26452],
       [    0,     0,     0, ...,     0,     0, 16664],
       ...,
       [    0,     0,     0, ...,   118,  1435,   168],
       [    0,     0,     0, ...,    16,  2029,   471],
       [    0,     0,     0, ...,   178,  6523,     1]])

In [161]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
le = preprocessing.LabelEncoder()

In [162]:
le.fit(y)

LabelEncoder()

In [163]:
list(le.classes_)

['Arabic',
 'Chinese',
 'Dutch',
 'English',
 'Estonian',
 'French',
 'Hindi',
 'Indonesian',
 'Japanese',
 'Korean',
 'Latin',
 'Persian',
 'Portugese',
 'Pushto',
 'Romanian',
 'Russian',
 'Spanish',
 'Swedish',
 'Tamil',
 'Thai',
 'Turkish',
 'Urdu']

In [164]:
y2=le.fit_transform(y)
total_languages = data['language'].nunique()
y2 = np_utils.to_categorical(y2,num_classes=total_languages)

In [165]:


X_train,X_test,Y_train,Y_test = train_test_split(train_data,y2)

embedding_dims = 500
vocab_size = len(tokenizer.word_index)+1

In [106]:
y2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [166]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(input_dim = vocab_size,output_dim = embedding_dims,input_length = max_len),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(total_languages,activation=tf.nn.softmax)
                            ])


model.compile(optimizer ='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [167]:
model.fit(X_train,Y_train)

516/516 [==============================] - 1855s 4s/step - loss: 1.3418 - accuracy: 0.6449


In [168]:
model_predict=model.predict(X_test)

In [181]:
text = ["エノが行きがかりでバスに乗ってしまい、気分が悪くなった際に助けるが、今すぐバスを降りたいと運転手に頼む際、本当のことを言ってしまうと彼女が恥ずかしい思いをすると察して「僕ウンコしたい"]
test_text = tokenizer.texts_to_sequences(text)
test_text = tf.keras.preprocessing.sequence.pad_sequences(test_text, maxlen=max_len)

In [182]:


predictions = model.predict(test_text)



In [183]:
out = predictions.argmax()
print(le.inverse_transform([out]))


['Chinese']
